13.07.2022

# Correzione manuale degli sparks

- generare automaticamente file excel/csv che contiene lista # events per ogni video con params:
    - `Movie ID`; `# event`; `duration`; `radius`; `# pixels`
    - File salvato in  `data/raw_data_and_processing/manual_corr_separated_events_masks/sparks_analysis.xlsx`
- ordinare gli eventi dal più grande al più piccolo e correggerli manualmente partendo dal più grande
- Load **XX_corrected_rgb_mask_V4.tif** and **XX_corrected_label_mask_V3.tif** and save as **XX_corrected_rgb_mask_SPARKS.tif** and **XX_corrected_label_mask_SPARKS.tif**

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import datetime
import imageio
import itertools
import napari
from matplotlib import cm
import pandas as pd

import numpy as np
import math
from PIL import Image
from scipy import spatial, optimize
import cc3d

from metrics_tools import (correspondences_precision_recall, 
                           nonmaxima_suppression, 
                           compute_filtered_butter, 
                           get_sparks_locations_from_mask,
                           process_spark_prediction
                          )
from dataset_tools import load_movies_ids, load_annotations_ids, load_rgb_annotations_ids

In [1]:
#movie_ids = ["01","02","03","04","05","06","07","08","09",
#             "10","11","12","13","14","15","16","17","18","19",
#             "20","21","22","23","24","25","27","28","29",
#             "30","32","33","34","35","36","38","39",
#             "40","41","42","43","44","45","46"
#            ]
movie_ids = ['07']

In [4]:
data_dir = os.path.join("..", "data", "raw_data_and_processing")

### directory where corrected masks are saved

In [5]:
# directory where corrected movies are saved
corr_dir = os.path.join(data_dir, "manual_corr_separated_event_masks")
os.makedirs(corr_dir, exist_ok=True)

## Save sparks list to excel file

### sparks list filename

In [6]:
#filename = 'sparks_analysis.xlsx'
filename = 'sparks_analysis_07.xlsx'

### function to analyse given spark

In [7]:
from scipy import ndimage

In [8]:
def analyse_spark_roi(spark_mask):
    '''
    spark_mask is the ROI denoting a spark event in the movie
    returns sparks duration, sparks max radius and number of pixels in the ROI
    '''
    # pixels
    locations = np.where(spark_mask)
    n_pixels = len(locations[0])
    
    # duration
    frames = np.unique(locations[0])
    duration = len(frames)
    
    # radius
    r = 0
    for f in frames:
        # calculate the distance of each foreground point to the closest background point
        dt = ndimage.distance_transform_edt(spark_mask[f])
        temp_r = np.amax(dt)
        if temp_r > r:
            r = temp_r
            
    return duration, r, n_pixels            

### for each movie, add list of file in pandas dataframe

In [9]:
# create pandas dataframe
cols = ['Movie ID','# event','duration','radius','# pixels']
sparks_df = pd.DataFrame(columns=cols)

for movie_id in movie_ids:
    print(movie_id)
    # open events mask and label mask
    events_mask = load_annotations_ids(data_folder=corr_dir, 
                                       ids=[movie_id], 
                                       mask_names="corrected_rgb_mask_V4")[movie_id]
    classes_mask = load_annotations_ids(data_folder=corr_dir, 
                                        ids=[movie_id], 
                                        mask_names="corrected_label_mask_V3")[movie_id]
    
    # get sparks mask
    sparks_event_mask = np.where(classes_mask==1, events_mask, 0)
    
    # get list of spark IDs
    spark_events_ids = list(np.unique(sparks_event_mask))
    spark_events_ids.remove(0)
    
    # add each spark to the dataframe
    for spark_id in spark_events_ids:
        spark_mask = sparks_event_mask == spark_id
        duration, radius, n_pixels = analyse_spark_roi(spark_mask)
        
        df = pd.DataFrame([[movie_id, spark_id, duration, radius, n_pixels]], 
                          columns=cols)
        sparks_df = sparks_df.append(df)
    
    

07


### save dataframe to excel file

In [10]:
sparks_df.to_excel(os.path.join(corr_dir,filename), index = False)

## REMARK: Filter out events that cannot be more than one spark

### Physiological params
- 1 pixel = 0.2 um x 0.2 um
- min distance in space between event centroids: 1.8 um
- 1 frame = 6.8 ms
- min distance in time between events: 20 ms

14.07.2022

## Correct manually events from largest to smallest

Starting from biggest event (in # pixels), correct them until it seems that the events contain only one spark.

Le modifiche sono registrate in  `data/raw_data_and_processing/manual_corr_separated_events_masks/sparks_analysis_PROCESSED.xlsx`

In [37]:
modified = ['01','02','03','04','05','06','07','08','09',
            '10','11','12','13','14','15','16','17','18','19',
            '20','21','22','23','24','25','27','28','29',
            '30','32','33','34','35','36','38','39',
            '40','41','42','43', '44','45','46']
len(modified)

43

In [12]:
# set smooth movies directory
smooth_movies_dir = os.path.join(data_dir, "smoothed_movies")

### General tools 

#### Create LUT for smooth movie

In [13]:
import vispy.color

# define function to obtain discrete Colormap instance that can be used by Napari

def get_discrete_cmap(name='viridis', lut=16):
    # create original cmap
    segmented_cmap = cm.get_cmap(name=name, lut=16)
    
    # get colors
    colors = segmented_cmap(np.arange(0,segmented_cmap.N)) 
    
    # get new discrete cmap
    cmap = vispy.color.Colormap(colors, interpolation='zero')
    
    return cmap

In [14]:
cmap = get_discrete_cmap(name='gray', lut=16)

#### Function to compute moving average

In [15]:
# Copied code from `"DEBUG Generate UNet masks.ipyinb"`.

def rolling_average(movie, k=3):
    '''
    movie: input movie
    k:     window size (must be an odd number)
    
    Return rolling average of movie along first axis (frames)
    '''
    
    # assert that k is an odd number
    assert k%2 == 1
    
    # compute number of frames to be considered on each side of given frame
    diff = k // 2
    
    # compute rolling average of input movie
    avg_movie = np.zeros_like(movie, dtype=float)
    avg_movie[:diff] = movie[:diff]
    avg_movie[-diff:] = movie[-diff:]

    for frame_idx in range(diff,len(movie)-diff):
        avg_movie[frame_idx] = np.sum(movie[frame_idx-diff : frame_idx+diff+1], axis=0)/k
    
    return avg_movie

#### General tools for wathershed algo

In [16]:
from scipy import ndimage as ndi

from skimage.feature import peak_local_max
from skimage.segmentation import watershed

In [17]:
# physiological params
PIXEL_SIZE = 0.2 # 1 pixel = 0.2 um x 0.2 um
global MIN_DIST_XY
MIN_DIST_XY = round(1.8 / PIXEL_SIZE) # min distance in space between sparks
TIME_FRAME = 6.8 # 1 frame = 6.8 ms
global MIN_DIST_T
MIN_DIST_T = round(20 / TIME_FRAME) # min distance in time between sparks

import math

# compute shape for maximum filter -> min distance between peaks
#min_dist = ellipsoid(MIN_DIST_T/2, MIN_DIST_XY/2, MIN_DIST_XY/2)
radius = math.ceil(MIN_DIST_XY/2)
y,x = np.ogrid[-radius: radius+1, -radius: radius+1]
disk = x**2+y**2 <= radius**2
min_dist_filter = np.stack([disk]*(MIN_DIST_T+1), axis=0)

In [18]:
def nonmaxima_suppression(img, min_dist, maxima_mask=None,
                          return_mask=False, threshold=0.5, sigma=2):
    '''
    Extract local maxima from input array (t,x,y).
    img :           input array
    maxima_mask :   if not None, look for local maxima only inside the mask
    min_dist :      mask representing minimal distance between local maxima
    return_mask :   if True return both masks with maxima and locations, if
                    False only returns locations
    threshold :     minimal value of maximum points
    sigma :         sigma parameter of gaussian filter
    '''
    img = img.astype(np.float64)
    
    if sigma > 0:
        if maxima_mask is not None:
            # keep only region inside mask
            masked_img = np.where(maxima_mask, img, 0.)

            # smooth masked input image
            smooth_img = ndi.gaussian_filter(masked_img, sigma=sigma)
        else:
            smooth_img = ndi.gaussian_filter(img, sigma=sigma)
    else:
        smooth_img = img
    
    if maxima_mask is not None:
        # hypothesis: maxima belong to maxima mask
        smooth_img = np.where(maxima_mask, smooth_img, 0.)

    # search for local maxima
    dilated = ndi.maximum_filter(smooth_img,
                                 footprint=min_dist)
    argmaxima = np.logical_and(smooth_img == dilated, smooth_img > threshold)

    argwhere = np.argwhere(argmaxima)

    if not return_mask:
        return argwhere

    return argwhere, argmaxima

In [19]:
# define function to split given spark DA VEDERE SE DA USARE O NO

def split_spark_event(movie, events_mask, event_id, n_sparks):
    
    # compute distance from edge of event
    event_mask = events_mask==event_id
    distances = ndi.distance_transform_edt(event_mask.astype(bool))
    
    coords = peak_local_max(distances, 
                            footprint=np.ones((3, 3, 3)), 
                            labels=events_mask.astype(bool))
    
    mask = np.zeros(distances.shape, dtype=bool)
    mask[tuple(coords.T)] = True
    
    markers, _ = ndi.label(mask)
    
    split_sparks_mask = watershed(#image=-movie,
                                  image=-distances,
                                  markers=markers,
                                  mask=events_mask==event_id,
                                  #markers=n_sparks
                                 )
    
    return split_sparks_mask

### Correct single event interactively on Napari

!!!! IF MOVIE HAS ALREADY BEEN PROCESSED, LOAD `_SPARK` VERSION OF MASKS !!!!

In [20]:
# select movie to be opened
movie_id = '07'

In [21]:
# open smooth movie, class and event labels
event_mask_name = "corrected_rgb_mask_"+("V4" if movie_id not in modified else "SPARKS")
classes_mask_name = "corrected_label_mask_"+("V3" if movie_id not in modified else "SPARKS")

movie = load_movies_ids(data_folder=smooth_movies_dir,
                        ids=[movie_id],
                        names_available=True,
                        movie_names="smoothed_video")[movie_id]

events_mask = load_annotations_ids(data_folder=corr_dir, 
                                   ids=[movie_id], 
                                   mask_names=event_mask_name
                                  )[movie_id]

classes_mask = load_annotations_ids(data_folder=corr_dir, 
                                    ids=[movie_id], 
                                    mask_names=classes_mask_name
                                   )[movie_id]

In [397]:
spark_ids = [4429331,
15758091,
16062453,
12225612,
4975264,
2934275,
13597925,
3623206]

In [34]:
# select event(s) to be opened
spark_id = 14726463
spark_ids = [spark_id]

#### Visualise smooth movie and event
Corrections have to be done on `events_mask` and `classes_mask`

In [35]:
# get sparks mask
sparks_event_mask = np.where(classes_mask==1, events_mask, 0)

# get list of spark IDs
spark_events_ids = list(np.unique(sparks_event_mask))
spark_events_ids.remove(0)

# get mask with sparks to be analised
spark_mask = np.zeros_like(sparks_event_mask)
for spark_id in spark_ids:
    spark_mask += np.where(sparks_event_mask == spark_id, sparks_event_mask, 0)

    print(f"frames containing event {spark_id}: {list(np.unique((np.where(sparks_event_mask == spark_id)[0])))}")

viewer = napari.Viewer()
viewer.add_image(movie, 
                 name='smooth movie', 
                 colormap=('colors',cmap))

viewer.add_labels(events_mask, 
                  name='events labels', 
                  opacity=0.5, 
                  visible=False)

viewer.add_labels(classes_mask, 
                  name='classes labels', 
                  opacity=0.5, 
                  visible=False)

viewer.add_labels(sparks_event_mask, 
                  name='sparks event mask', 
                  opacity=0.2, 
                  visible=True)

viewer.add_labels(spark_mask, 
                  name='event to analyse', opacity=0.6, 
                  visible=True)

frames containing event 14726463: [434, 435, 436]


<Labels layer 'event to analyse' at 0x13c916efca0>

#### Run this to save new masks

In [36]:
imageio.volwrite(os.path.join(corr_dir, movie_id+"_corrected_rgb_mask_SPARKS.tif"), 
                 events_mask)
imageio.volwrite(os.path.join(corr_dir, movie_id+"_corrected_label_mask_SPARKS.tif"), 
                 classes_mask)

modified.append(movie_id)
print("Modified movies:", modified)

Modified movies: ['01', '02', '03', '04', '05', '06', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '27', '28', '29', '30', '32', '33', '34', '35', '36', '38', '39', '40', '41', '42', '43', '44', '45', '46', '07']


#### Run this to change class of a # event

In [314]:
n_event = 8
new_class = 4

classes_mask = np.where(events_mask == n_event , new_class, classes_mask)

#### Run this to get list of events

In [399]:
print("Movie ID", movie_id)
list_events = list(np.unique(events_mask))
list_events.remove(0)
print(list_events)

Movie ID 11
[5, 6, 7, 2294, 161952, 1132609, 1693437, 2934275, 2936115, 3623206, 4383668, 4429331, 4975264, 5176013, 5549575, 5873836, 7266172, 7547622, 7564447, 8864374, 8906090, 9725835, 10776718, 12225612, 12864277, 12995516, 13208235, 13597925, 15326058, 15758091, 16062453, 16376841]


#### Run this to get smoother version of smooth movie

In [402]:
smoother_movie = rolling_average(movie, k=3)

In [297]:
imageio.volwrite("TEST_SMOOTH.tif", smoother_movie)

In [403]:
viewer.add_image(smoother_movie, 
                 name='smoother movie', 
                 colormap=('colors',cmap))


<Image layer 'smoother movie' at 0x11ad3bc1910>

### Run this to separate given event using watershed segmentation

#### Compute distance from event edge

In [575]:
distances = ndi.distance_transform_edt(spark_mask.astype(bool))

#### Compute markers with nonmaxima suppression

In [570]:
# can try this instead of nonmaxima_suppression
# need first to smooth movie
'''coords = peak_local_max(distances, 
                        footprint=min_dist_filter, 
                        labels=spark_mask.astype(bool))''';

In [355]:
coords, max_mask = nonmaxima_suppression(img=movie,
                                         #img=smoother_movie,
                                         min_dist=min_dist_filter,
                                         maxima_mask=spark_mask,
                                         return_mask=True,
                                         threshold=0.,
                                         sigma=1)

print("Sparks peaks found:", coords)

markers, _ = ndi.label(max_mask)

Sparks peaks found: [[428  36 349]
 [434  35 346]
 [438  35 344]]


#### Select markers manually

In [358]:
coords = np.array([[428,  36, 349],
                   [438,  35, 344],
                  ]) # [z,y,x]

mask = np.zeros(movie.shape, dtype=bool)
mask[tuple(coords.T)] = True
markers, _ = ndi.label(mask)

#### Apply watershed fct

In [359]:
watershed_mask = watershed(image=-movie, # use input movie
                           #image=-smoother_movie, # use smoother input movie
                           #image=movie.astype(bool), # use binary image
                           #image=-distances, # use distance from edge
                           markers=markers,
                           mask=spark_mask)

#### Visualise separated event

In [360]:
viewer.add_labels(watershed_mask, 
                  name='watershed mask', opacity=0.5, 
                  visible=True)

<Labels layer 'watershed mask [1]' at 0x119bee7a370>

#### Run this to assign separated events in watershed mask to `events_mask`

In [363]:
n_watershed_event = 2
new_event = 7

events_mask = np.where(watershed_mask == n_watershed_event , new_event, events_mask)

#### Run this to change # to an event

In [323]:
old_event = 16
new_event = 5472814

events_mask = np.where(events_mask == old_event , 
                       new_event, 
                       events_mask)